In [113]:
import sys
import time

import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql
from sqlalchemy import create_engine

from __future__ import print_function, division

import pandas_datareader as pdr

import requests
import random
from bs4 import BeautifulSoup

import pickle

import html5lib

from fake_useragent import UserAgent

In [22]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}

In [36]:
response = requests.get('https://finance.yahoo.com/quote/UWT/risk?p=UWT', headers=user_agent)

In [37]:
soup = BeautifulSoup(response.text, 'lxml')

In [38]:
span_list = ['31','35','39','46','50','54','61','65','69','76','80','84','91',
             '95','99','106','110','114','121','125','129']

risk = soup.find_all('span', {'data-reactid': span_list})[4:]

In [39]:
risk

[<span class="W(39%) Fl(start)" data-reactid="31">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="35">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="39">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="46">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="50">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="54">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="61">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="65">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="69">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="76">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="80">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="84">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="91">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="95">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="99">0</span>,
 <span class="W(39%) Fl(start)" data-reactid="106">0</span>,
 <span class="W(39%) Fl(start)" data-re

In [23]:
risk_df = pd.DataFrame(columns = {'3 Year Alpha', '5 Year Alpha', '10 Year Alpha',
                                  '3 Year Beta', '5 Year Beta', '10 Year Beta',
                                  '3 Year Mean Annual Return', '5 Year Mean Annual Return', '10 Year Mean Annual Return',
                                  '3 Year R-Squared', '5 Year R-Squared', '10 Year R-Squared',
                                  '3 Year Standard Deviation', '5 Year Standard Deviation', '10 Year Standard Deviation',
                                  '3 Year Sharpe Ratio', '5 Year Sharpe Ratio', '10 Year Sharpe Ratio',
                                  '3 Year Treynor Ratio', '5 Year Treynor Ratio', '10 Year Treynor Ratio'})

In [24]:
with open('tickers.pkl', 'rb') as f:
    tickers = pickle.load(f)

In [48]:
risk_data = []

In [49]:
len(tickers)

2007

In [65]:
symbols = pdr.get_nasdaq_symbols()

In [67]:
etfs = symbols[symbols['ETF'] == True]
etfs = etfs.reset_index()

In [68]:
tickers = etfs['Symbol'].tolist()

In [69]:
len(tickers)

2093

In [70]:
span_list = ['31','35','39','46','50','54','61','65','69','76','80','84','91',
             '95','99','106','110','114','121','125','129']

In [71]:
def parse_risk_data(ticker):
    response = requests.get('https://finance.yahoo.com/quote/{}/risk?p={}'.format(ticker,ticker), headers=user_agent)
    soup = BeautifulSoup(response.text, 'lxml')
    
    risk = soup.find_all('span', {'data-reactid': span_list})[4:]
    
    risk_list = []
    
    for data in risk:
        risk_list.append(data.text)
    '''
    risk_data['3 Year Alpha'] = risk[0].text
    risk_data['5 Year Alpha'] = risk[1].text
    risk_data['10 Year Alpha'] = risk[2].text
    risk_data['3 Year Beta'] = risk[3].text
    risk_data['5 Year Beta'] = risk[4].text
    risk_data['10 Year Beta'] = risk[5].text
    risk_data['3 Year Mean Annual Return'] = risk[6].text
    risk_data['5 Year Mean Annual Return'] = risk[7].text
    risk_data['10 Year Mean Annual Return'] = risk[8].text
    risk_data['3 Year R-Squared'] = risk[9].text
    risk_data['5 Year R-Squared'] = risk[10].text
    risk_data['10 Year R-Squared'] = risk[11].text
    risk_data['3 Year Standard Deviation'] = risk[12].text
    risk_data['5 Year Standard Deviation'] = risk[13].text
    risk_data['10 Year Standard Deviation'] = risk[14].text
    risk_data['3 Year Sharpe Ratio'] = risk[15].text
    risk_data['5 Year Sharpe Ratio'] = risk[16].text
    risk_data['10 Year Sharpe Ratio'] = risk[17].text
    risk_data['3 Year Treynor Ratio'] = risk[18].text
    risk_data['5 Year Treynor Ratio'] = risk[19].text
    risk_data['10 Year Treynor Ratio'] = risk[20].text'''
    
    return risk_list

In [72]:
counter = 0

for ticker in tickers:
    risk_list = parse_risk_data(ticker)
    risk_data.append(risk_list)
    print(counter)
    counter += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [74]:
risk_data_copy = risk_data[2007:]

In [75]:
risk_data = risk_data_copy

len(risk_data)

2093

In [90]:
for index, risk in enumerate(risk_data):
    if len(risk) >= 21:
        print(index, tickers[index], len(risk), risk_data[index][0],risk_data[index][1],risk_data[index][2])

0 AAAU 21 0 0 0
1 AADR 21 -2.31 2.55 0
2 AAXJ 21 2.8 3.18 1.48
3 ACES 21 0 0 0
4 ACIM 21 3.19 4.35 0
5 ACSG 21 0 0 0
6 ACSI 21 0 0 0
7 ACT 21 0 0 0
8 ACWF 21 2.57 0 0
9 ACWI 21 3.34 4.12 4.04
10 ACWV 21 3.6 6.93 0
11 ACWX 21 -0.12 -0.07 -0.23
12 ADRA 21 1.15 1.89 0.24
13 ADRD 21 -1.15 -1.94 -0.9
14 ADRE 21 2.82 2.6 -2.56
15 ADRU 21 -1.73 -2.25 -0.87
16 AFIF 21 0 0 0
17 AFK 21 -0.08 -6.64 -3.94
18 AFTY 21 0 0 1.24
19 AGG 21 -0.05 -0.05 -0.14
20 AGGE 21 0 0 0
21 AGGP 21 0 0 0
22 AGGY 21 0.7 0 0
23 AGND 21 2.06 0.61 0
24 AGQ 22 -1.73 -8.74 6.55
25 AGT 21 0 0 0
26 AGZ 21 -0.48 -0.13 -0.39
27 AGZD 21 1.33 0.47 0
28 AIA 21 5.9 5.63 3.31
29 AIEQ 21 0 0 0
30 AIIQ 21 N/A N/A N/A
31 AIQ 21 0 0 0
32 AIRR 21 1.98 0 0
33 ALD 21 1.3 -1.51 0
34 ALFA 21 -2.58 -4.45 N/A
35 ALTS 21 -3.32 0 0
36 ALTY 21 -0.99 0 0
37 AMCA 21 0 0 0
38 AMLP 21 -7.28 -10.48 0
39 AMZA 21 -8.86 0 0
40 ANGL 21 9.86 4.86 0
41 AOA 21 -0.83 -0.42 -1.13
42 AOIL 21 N/A N/A N/A
43 AOK 21 -0.45 0.14 0.9
44 AOM 21 -0.51 0.03 0.26
45 AO

In [77]:
tickers[0]

'AAAU'

In [94]:
type(clean_etfs[60][0])

int

In [98]:
full_length = []
tickers_keep = []


for index, etf in enumerate(risk_data):
    if len(etf) >= 21:
        full_length.append(etf)
        tickers_keep.append(tickers[index])
    else:
        pass

In [100]:
len(tickers_keep)

2086

In [107]:
clean_etfs = []

for index, etf in enumerate(full_length):
    append_list = []
    for stat in etf:
        stat_str = str(stat)
        if stat_str == 'N/A':
            append_list.append(0)
        else:
            append_list.append(stat_str)
    clean_etfs.append(append_list)

In [108]:
full_etfs = []
tickers_to_keep = []

for index, etf in enumerate(clean_etfs):
    append_list = []
    for stat in etf:
        try:
            append_list.append(float(stat))
        except:
            print(stat)
            
    if len(append_list) >= 21:
        full_etfs.append(append_list)
        tickers_to_keep.append(tickers_keep[index])
    else:
        pass
    float_etfs.append(append_list)





7,765.64

































































1,347.12




















-3,618.15



































In [110]:
len(tickers_to_keep)

2082

In [111]:
for etf in full_etfs:
    if len(etf) > 21:
        if etf[1] != 0 and etf[0] > 3 * np.abs(float(etf[1])):
            etf.pop(0)

In [112]:
for index, risk in enumerate(full_etfs):
    print(index, tickers_to_keep[index], len(risk), full_etfs[index][0],full_etfs[index][1],full_etfs[index][2])

0 AAAU 21 0.0 0.0 0.0
1 AADR 21 -2.31 2.55 0.0
2 AAXJ 21 2.8 3.18 1.48
3 ACES 21 0.0 0.0 0.0
4 ACIM 21 3.19 4.35 0.0
5 ACSG 21 0.0 0.0 0.0
6 ACSI 21 0.0 0.0 0.0
7 ACT 21 0.0 0.0 0.0
8 ACWF 21 2.57 0.0 0.0
9 ACWI 21 3.34 4.12 4.04
10 ACWV 21 3.6 6.93 0.0
11 ACWX 21 -0.12 -0.07 -0.23
12 ADRA 21 1.15 1.89 0.24
13 ADRD 21 -1.15 -1.94 -0.9
14 ADRE 21 2.82 2.6 -2.56
15 ADRU 21 -1.73 -2.25 -0.87
16 AFIF 21 0.0 0.0 0.0
17 AFK 21 -0.08 -6.64 -3.94
18 AGG 21 -0.05 -0.05 -0.14
19 AGGE 21 0.0 0.0 0.0
20 AGGP 21 0.0 0.0 0.0
21 AGGY 21 0.7 0.0 0.0
22 AGND 21 2.06 0.61 0.0
23 AGQ 21 -1.73 -8.74 6.55
24 AGT 21 0.0 0.0 0.0
25 AGZ 21 -0.48 -0.13 -0.39
26 AGZD 21 1.33 0.47 0.0
27 AIA 21 5.9 5.63 3.31
28 AIEQ 21 0.0 0.0 0.0
29 AIIQ 21 0.0 0.0 0.0
30 AIQ 21 0.0 0.0 0.0
31 AIRR 21 1.98 0.0 0.0
32 ALD 21 1.3 -1.51 0.0
33 ALFA 21 -2.58 -4.45 0.0
34 ALTS 21 -3.32 0.0 0.0
35 ALTY 21 -0.99 0.0 0.0
36 AMCA 21 0.0 0.0 0.0
37 AMLP 21 -7.28 -10.48 0.0
38 AMZA 21 -8.86 0.0 0.0
39 ANGL 21 9.86 4.86 0.0
40 AOA 21 -0.83

In [114]:
for etf in full_etfs:
    if len(etf) > 21:
        etf.pop()

In [117]:
len(tickers_to_keep)

2082

In [125]:
cols = list(range(21))

risk_df = pd.DataFrame(columns = cols)

In [126]:
for index, etf in enumerate(full_etfs):
    risk_df.loc[tickers_to_keep[index]] = etf

In [127]:
risk_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
AAAU,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AADR,-2.31,2.55,0.00,1.21,1.03,0.00,0.88,0.48,0.00,64.11,...,0.00,17.18,15.26,0.00,0.54,0.33,0.00,6.83,3.87,0.00
AAXJ,2.80,3.18,1.48,1.10,1.06,1.05,1.22,0.54,1.04,76.96,...,80.88,14.30,14.73,18.59,0.93,0.39,0.65,12.08,4.53,10.36
ACES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACIM,3.19,4.35,0.00,0.87,0.86,0.00,1.07,0.60,0.00,85.20,...,0.00,10.72,11.02,0.00,1.07,0.58,0.00,13.47,7.00,0.00
ACSG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACSI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACWF,2.57,0.00,0.00,0.94,0.00,0.00,1.07,0.00,0.00,82.96,...,0.00,11.72,0.00,0.00,0.98,0.00,0.00,12.40,0.00,0.00
ACWI,3.34,4.12,4.04,0.87,0.87,0.87,1.08,0.58,1.10,87.01,...,94.61,10.67,11.05,14.20,1.10,0.56,0.90,13.66,6.67,14.37


In [128]:
risk_df.columns = ['3 Year Alpha', '5 Year Alpha', '10 Year Alpha',
                   '3 Year Beta', '5 Year Beta', '10 Year Beta',
                   '3 Year Mean Annual Return', '5 Year Mean Annual Return', '10 Year Mean Annual Return',
                   '3 Year R-Squared', '5 Year R-Squared', '10 Year R-Squared',
                   '3 Year Standard Deviation', '5 Year Standard Deviation', '10 Year Standard Deviation',
                   '3 Year Sharpe Ratio', '5 Year Sharpe Ratio', '10 Year Sharpe Ratio',
                   '3 Year Treynor Ratio', '5 Year Treynor Ratio', '10 Year Treynor Ratio']

In [129]:
risk_df

,3 Year Alpha,5 Year Alpha,10 Year Alpha,3 Year Beta,5 Year Beta,10 Year Beta,3 Year Mean Annual Return,5 Year Mean Annual Return,10 Year Mean Annual Return,3 Year R-Squared,...,10 Year R-Squared,3 Year Standard Deviation,5 Year Standard Deviation,10 Year Standard Deviation,3 Year Sharpe Ratio,5 Year Sharpe Ratio,10 Year Sharpe Ratio,3 Year Treynor Ratio,5 Year Treynor Ratio,10 Year Treynor Ratio
AAAU,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AADR,-2.31,2.55,0.00,1.21,1.03,0.00,0.88,0.48,0.00,64.11,...,0.00,17.18,15.26,0.00,0.54,0.33,0.00,6.83,3.87,0.00
AAXJ,2.80,3.18,1.48,1.10,1.06,1.05,1.22,0.54,1.04,76.96,...,80.88,14.30,14.73,18.59,0.93,0.39,0.65,12.08,4.53,10.36
ACES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACIM,3.19,4.35,0.00,0.87,0.86,0.00,1.07,0.60,0.00,85.20,...,0.00,10.72,11.02,0.00,1.07,0.58,0.00,13.47,7.00,0.00
ACSG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACSI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ACWF,2.57,0.00,0.00,0.94,0.00,0.00,1.07,0.00,0.00,82.96,...,0.00,11.72,0.00,0.00,0.98,0.00,0.00,12.40,0.00,0.00
ACWI,3.34,4.12,4.04,0.87,0.87,0.87,1.08,0.58,1.10,87.01,...,94.61,10.67,11.05,14.20,1.10,0.56,0.90,13.66,6.67,14.37


In [132]:
engine = create_engine('postgres://ubuntu@18.218.14.250:5432/roboadvisor')

In [133]:
risk_df.to_sql('risk_metrics', engine, if_exists = 'replace')

In [134]:
len(risk_df)

2082